In [1]:
import os;
import tensorflow as tf
os.environ['PYTHONHASHSEED']=str(7)

# Reproduce the results
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(7)
   #np.random.seed(7)
   #random.seed(7)
   tf.random.set_seed(7)


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk import tokenize, stem
from tensorflow.keras.preprocessing.text import Tokenizer
import string
nltk.download("stopwords")
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Flatten, Activation, Dense, LSTM, BatchNormalization, Embedding, Dropout, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.optimizers import SGD

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score, roc_auc_score
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten


import pickle as pkl
from tensorflow.keras.models import Model, Sequential

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import re

#STOPWORDS = set(stopwords.words('english'))

win = tk.Tk()
win.title('Industrial Safety NLP based Chatbot')  # Window Title

# Step 1: Import data frame name ---------------------------------

Name=ttk.Label(win,text="File Name")
Name.grid(row=0,column=0,sticky=tk.W)

Name_var=tk.StringVar()
Name_entrybox=ttk.Entry(win,width=16,textvariable=Name_var)
Name_entrybox.grid(row=0,column=1)

def Import_Data():
    global DB
    DF_Name=Name_var.get()
    if len(DF_Name)==0:
        confirm="File Not Present"
    DB_extension=re.findall("\..*", DF_Name) 
    if DB_extension==['.xlsx']:
        DB=pd.read_excel(DF_Name)
        confirm="Done"
    elif DB_extension==['.csv']:
        DB=pd.read_csv(DF_Name)
        confirm="Done"
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=0,column=3)
    Confirm_entrybox.insert(1,str(confirm))   

Import_Data_Button=ttk.Button(win,text="Import Data",command=Import_Data)
Import_Data_Button.grid(row=0,column=2)


# Step 2: Target data frame name ---------------------------------

Target=ttk.Label(win,text="Target Colummn")
Target.grid(row=1,column=0,sticky=tk.W)

Target_var=tk.StringVar()
Target_entrybox=ttk.Entry(win,width=16,textvariable=Target_var)
Target_entrybox.grid(row=1,column=1)

def Target_Data():
    global DB, Target_Name
    Target_Name=Target_var.get()
    
    Column_name = DB.columns
    Column_name
    found=0

    for i in range(len(Column_name)):
        if Column_name[i]==Target_Name:
            confirm="Found"
        else:
            confirm="Not Found"
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=1,column=3)
    Confirm_entrybox.insert(1,str(confirm))


Target_Button=ttk.Button(win,text="Import Target",command=Target_Data)
Target_Button.grid(row=1,column=2)

## Step 3: pre-processing  ---------------------------------

process=ttk.Label(win,text="Pre-processing")
process.grid(row=2,column=0,sticky=tk.W)

global text


def remove_punctuation(text):
    PUNCT_TO_REMOVE = string.punctuation
    return text.translate(str.maketrans('','',PUNCT_TO_REMOVE))

def remove_stopwords(text):
    STOPWORDS = set(stopwords.words('english'))
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
def clean_text(text):
    text = text.strip()
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = text.lower()
    return text

def preprocess():
    global DB
    dict1 = {'I':1, 'II':2,'III':3,'IV':4, 'V':5}
    DB['Accident Level'] = DB['Accident Level'].map(dict1)
    
    
    text = DB['Description']
    text= text.str.lower()                                 
    text = text.apply(lambda text: remove_punctuation(text))
    text = text.apply(lambda text : remove_stopwords(text))    
    DB['Description'] = DB['Description'].map(lambda text : clean_text(text))
    DB['Description'].replace('', np.nan, inplace = True)
    DB = DB.dropna()
    
    
    DB= DB.drop(['Data','Unnamed: 0'],axis=1)
    confirm="Done"
    Confirm_entrybox=ttk.Entry(win,width=20)
    Confirm_entrybox.grid(row=2,column=3)
    Confirm_entrybox.insert(1,str(confirm))
    
    return DB

process_button=ttk.Button(win,text="Pre-process",command=preprocess)
process_button.grid(row=2,column=2)


def Embed():
    global DB, x_new1, y_new1, embedding_size, embedding_matrix, num_words,maxlen,vocab_size,X_train, X_test, y_train, y_test
    
    x_new1 = DB['Description']
    y_new1 = DB['Accident Level']
    
    
    X_train, X_test, y_train, y_test = train_test_split(x_new1, y_new1, test_size = 0.20, random_state = 1, stratify = y_new1)
    
    y_train = pd.get_dummies(y_train).values
    y_test = pd.get_dummies(y_test).values
    #y_train = to_categorical(y_train)
    #y_test = to_categorical(y_test)
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)
    vocab_size = len(tokenizer.word_index) + 1
    print("vocab_size:", vocab_size)

    maxlen = 100

    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

    embedding_size = 200
    embeddings_dictionary = dict()

    glove_file = open('glove.6B.200d.txt', encoding="utf8")

    for line in glove_file:
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary[word] = vector_dimensions

    glove_file.close()

    embedding_matrix = np.zeros((vocab_size, embedding_size))

    for word, index in tokenizer.word_index.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            
    confirm="Done"        
    Confirm_entrybox=ttk.Entry(win,width=20)
    Confirm_entrybox.grid(row=3,column=3)
    Confirm_entrybox.insert(1,str(confirm))
    
    
embed_button=ttk.Button(win,text="Embedding",command=Embed)
embed_button.grid(row=3,column=2)

model1=ttk.Label(win,text="Model predict")
model1.grid(row=4,column=0,sticky=tk.W)   

def model_best():    
    global maxlen,vocab_size,embedding_size,embedding_matrix,X_train, y_train,X_test, y_test, model
    reset_random_seeds()
    
    # Build a LSTM Neural Network
    deep_inputs = Input(shape=(maxlen,))
    embedding_layer = Embedding(vocab_size, embedding_size, weights=[embedding_matrix], trainable=False)(deep_inputs)
    LSTM_Layer_1 = Bidirectional(LSTM(128, return_sequences = True))(embedding_layer)
    max_pool_layer_1 = GlobalMaxPool1D()(LSTM_Layer_1)
    drop_out_layer_1 = Dropout(0.5, input_shape = (256,))(max_pool_layer_1)
    dense_layer_1 = Dense(128, activation = 'relu')(drop_out_layer_1)
    drop_out_layer_2 = Dropout(0.5, input_shape = (128,))(dense_layer_1)
    dense_layer_2 = Dense(64, activation = 'relu')(drop_out_layer_2)
    drop_out_layer_3 = Dropout(0.5, input_shape = (64,))(dense_layer_2)

    dense_layer_3 = Dense(32, activation = 'relu')(drop_out_layer_3)
    drop_out_layer_4 = Dropout(0.5, input_shape = (32,))(dense_layer_3)

    dense_layer_4 = Dense(10, activation = 'relu')(drop_out_layer_4)
    drop_out_layer_5 = Dropout(0.5, input_shape = (10,))(dense_layer_4)

    dense_layer_5 = Dense(5, activation='softmax')(drop_out_layer_5)
    dense_layer_3 = Dense(5, activation='softmax')(drop_out_layer_3)

    LSTM_Layer_1 = LSTM(128)(embedding_layer)
    dense_layer_1 = Dense(5, activation='softmax')(LSTM_Layer_1)
    
    model = Model(inputs=deep_inputs, outputs=dense_layer_1)
    model = Model(inputs=deep_inputs, outputs=dense_layer_5)
    model = Model(inputs=deep_inputs, outputs=dense_layer_3)

    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    # fit the keras model on the dataset
    print(X_train.shape, y_train.shape,X_test.shape, y_test.shape)
    training_history = model.fit(X_train, y_train, epochs=10, batch_size=8, verbose=1, validation_data=(X_test, y_test))
    training_history_DB = pd.DataFrame(training_history.history)
    
    accuracy_field = ttk.Entry(win, width=20)
    accuracy_field.grid(row=6, column=2)
    accuracy_field.insert(1, str(training_history.history['accuracy'][-1]))
    
    val_accuracy_field = ttk.Entry(win, width=20)
    val_accuracy_field.grid(row=6, column=3)
    val_accuracy_field.insert(1, str(training_history.history['val_accuracy'][-1]))

final_button = ttk.Button(win, text = "Training", command = model_best)  
final_button.grid(row=6, column=1)

def pkl():
    global model
    model.save('BiLSTM_model123.h5')
    model.save_weights('BiLSTM_model_weights.h5')
    confirm = "model is saved"
    model_accuracy = ttk.Entry(win, width=20)
    model_accuracy.grid(row=7, column=2)
    model_accuracy.insert(1, str(confirm))
    
final_pickle = ttk.Button(win, text="Pickle", command=pkl)
final_pickle.grid(row=7, column=1)


Description_var = tk.StringVar()
Description_entrybox = ttk.Entry(win,width=20,textvariable=Description_var)
Description_entrybox.grid(row=9,column=0) 


def predict_chat():
    global sentence,X_train
    
   
    sentence = Description_var.get()
    sentence = sentence.lower()                                 
    sentence = remove_punctuation(sentence)
    sentence =  remove_stopwords(sentence)    
    sentence =  clean_text(sentence)
    
    tokenizer = Tokenizer(num_words=5000)
    #tokenizer.fit_on_texts(X_train)
    sentence = tokenizer.texts_to_sequences(sentence)
    sentence = pad_sequences(sentence, padding='post', maxlen=25)
    
    result = model.predict(sentence)
    predicted_index = np.argmax(result[0])
    mapping = {1:'Level 1', 2:'Level 2', 3:'Level 3'}
    predicted = mapping[predicted_index]
    predicted
    
    confirm = "The level is : "+ predicted
    confirm_pred = ttk.Entry(win, width=20)
    confirm_pred.grid(row=8, column=2)
    confirm_pred.insert(1, str(confirm))
    
predicted_button = ttk.Button(win, text="Output", command = predict_chat)
predicted_button.grid(rows=8,column=1)
    
win.mainloop()

vocab_size: 3118
(340, 100) (340, 5) (85, 100) (85, 5)
Epoch 1/10
43/43 [==============================] - 6s 93ms/step - loss: 1.5199 - accuracy: 0.3666 - val_loss: 0.9900 - val_accuracy: 0.7412
Epoch 2/10
43/43 [==============================] - 4s 86ms/step - loss: 1.0926 - accuracy: 0.7442 - val_loss: 0.9432 - val_accuracy: 0.7412
Epoch 3/10
43/43 [==============================] - 4s 87ms/step - loss: 1.0008 - accuracy: 0.7460 - val_loss: 0.9424 - val_accuracy: 0.7412
Epoch 4/10
43/43 [==============================] - 4s 87ms/step - loss: 1.0376 - accuracy: 0.7441 - val_loss: 0.9476 - val_accuracy: 0.7412
Epoch 5/10
43/43 [==============================] - 4s 86ms/step - loss: 1.0724 - accuracy: 0.7291 - val_loss: 0.9430 - val_accuracy: 0.7412
Epoch 6/10
43/43 [==============================] - 4s 86ms/step - loss: 0.9268 - accuracy: 0.7666 - val_loss: 0.9414 - val_accuracy: 0.7412
Epoch 7/10
43/43 [==============================] - 4s 88ms/step - loss: 1.0116 - accuracy: 0.7346 

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Admin\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-8-4488ced977a4>", line 260, in predict_chat
    result = model.predict(sentence)
  File "C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1629, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 726, in _initialize
    *args, **kwds))
  File "C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\eager\f